In [29]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        predicted_class = torch.where(out > 0.5, torch.tensor(0.0), torch.tensor(1.0))

        return predicted_class

input_size = 2
hidden_size = 32
num_layers = 10
output_size = 1

model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)


In [33]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

data_dir = '../data/emg_rms/'

class CustomDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(data_dir, self.file_list[idx])
        df = pd.read_csv(file_path)

        features = df[['V1', 'V2']].values
        label = df[df.columns[-1]].loc[0]

        return torch.FloatTensor(features), label


In [ ]:
file_list = os.listdir(data_dir)

train_files, val_files = train_test_split(file_list, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_files)
val_dataset = CustomDataset(val_files)

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs[0], labels)
        print(loss)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        total_correct = 0
        total_samples = 0

        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs[0], labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        average_val_loss = val_loss / len(val_loader)
        accuracy = total_correct / total_samples

        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}')